# Packages

In [1]:
import sys
import utils as ut
import json
import re

# Input

In [7]:
def DAT(clist):
    i = 1
    while clist:
        if i in [1, 4, 5]:
            clist.pop(0)
            pass
        elif i == 2:
            JOBNAME, MSGL = ut.tokenizer(clist.pop(0))
            MSGL = int(MSGL)
        elif i == 3:
            [FOR001] = ut.tokenizer(clist.pop(0))
        elif i == 6:
            Lmax, NSR, NFI = list(map(int, ut.tokenizer(clist.pop(0))))
        elif i == 7:
            NPRN, IVEF = list(map(int, ut.tokenizer(clist.pop(0))))
        else:
            print("clist exist")
            clist.pop(0)
        i+=1
    return [JOBNAME, MSGL, FOR001, NPRN, Lmax, NSR, NFI, IVEF]


In [11]:
# Read the file and parse a text
filename="lat_bcc.dat"
with open(filename) as f:
    lines = f.readlines()

# cleaning & get list type [line, line, ...]
clist = ut.Cleaning(lines)

# extract info.
DAT_values = DAT(clist)
print(DAT_values)

# define names and units of info.
DAT_keys = ['JOBNAME', 'MSGL', 'FOR001', 'NPRN', 'Lmax', 'NSR', 'NFI', 'IVEF']
DAT_units = [None, None, None, None, None, None, None, None]

# add unit ot info.
DAT_vu = [[value, unit] for value, unit in zip(DAT_values, DAT_units)]

# add name to info. and divide info. into sub-classes
Meta_dict = {key:value for key, value in zip(DAT_keys[0:4], DAT_vu[0:4])}
Approximation_dict = {key:value for key, value in zip(DAT_keys[4:], DAT_vu[4:])}

# combine all
DAT_dict = {'Meta':Meta_dict, 'Approximation':Approximation_dict}

# save as json file
with open(filename+'_in.json', 'w') as f:
    json.dump(DAT_dict, f, indent=2)

['lat_bcc', 1, '../../kstr/lat_bcc/smx/lat_bcc.tfh', 0, 30, 129, 11, 3]


# Output

In [14]:
# read a file
filename = "lat_bcc.prn"
with open(filename) as f:
    lines = f.readlines()
# convert file to compact list
clist = ut.Cleaning(lines)

In [16]:
# input clist
ut.check_dat(clist.pop(0), 'SHAPE')

 ### Error : keyword mismatch; keyword = SHAPE, input = FHND



SystemExit: 

/home/tjsrb2507/anaconda3/envs/pyemto/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
def FHNDLR1(f):
    # read time
    timestamp = ut.head_timestamp(f)
    return timestamp  ### 내용 리스트 내보내기
def FHNDLR2(f):
    # read FOR001
    FOR001 = ut.css(f)
    # flush 2 lines (FOR002, FOOR006)
    flush(f,2)
    return FOR001
def FHNDLR3(f):
    # read system information
    EMTO = ut.css(f)
    branch = ut.css(f)
    hash_key = ut.css(f)
    compile_on = ut.css(f)
    OS = ut.css(f)
    CPU = ut.css(f)
    compiler = ut.css(f)
    library = ut.css(f)
    return EMTO, branch, hash_key, compile_on, OS, CPU, compiler, library
def INPUT(f):
    # NSR, LMAX, NFI, IVEF
    NSR, LMAX, NFI, IVEF = ut.getint(f)
    return NSR, LMAX, NFI, IVEF
def TRNSFM(f):
    # slope_matrices, KW2
    string = f.readline()
    slope_matrices = string[26:36].strip()
    KW2 = float(string[50:60])
    return slope_matrices, KW2
def BLATTS(f):
    # format {plane_num: 1, plane_info:[x,y,z,d], point_num: 6, point_info:[[x,y,z,d(상대값)],...]}
    site_info = []
    while 1:
        string = f.readline()
        # strip first line
        if "V(tetra)" in string:
            print(string)
            site_info.append(plane_info)
            v_info = {}
            str_list = ut.getstring_withoutequal(f, string)
            float_list = ut.getfloat(f, string)
            v_info[str_list[0]] = float_list[0]
            v_info[str_list[1]] = float_list[1]
            site_info.append(v_info)
            break
        elif len(string.split()) == 6:
            try:
                site_info.append(plane_info)
            except:
                pass
            plane_info = {}
            point_info = []
            str_list = string.split()
            plane_info['plane_num'] = int(str_list[0])
            plane_info['plane_info'] = [float(str_list[1]), float(str_list[2]), float(str_list[3]), float(str_list[4])]
            plane_info['point_num'] = int(str_list[5])
            
        elif len(string.split()) == 4:
            point_info.append([float(string.split()[0]), float(string.split()[1]), float(string.split()[2]), float(string.split()[3])])
            plane_info['point_info'] = point_info
            
    return site_info

def RMESH(f):
    # format {NINT: 4, RINT: [1,1,1,1], NSRI: [1,1,1], DSRI: [1,1,1]}
    rmesh_dict = {}
    rmesh_dict['NINT'] = ut.getint(f)[0]
    
    temp_list = []
    for i in range(rmesh_dict['NINT']):
        temp_list.append(ut.e1f(f))
    rmesh_dict['RINT'] = temp_list    
    
    temp_list1 = [] # for NSRI
    temp_list2 = [] # for DSRI
    for i in range(rmesh_dict['NINT']-1): ### Error 처리
        sri_list = ut.getfloat(f)
        temp_list1.append(int(sri_list[1]))
        temp_list2.append(sri_list[3])
    rmesh_dict['NSRI'] = temp_list1
    rmesh_dict['DSRI'] = temp_list2
    return rmesh_dict
    
def SETROT(f):
    # format 2
    setrot = ut.getint(f)[0]
    return setrot
    
def UPDATE(f): ## [l, d(l), D(l), %], [0, 0.6, 0.65, 34.5] 이렇게 수정
    # format {l:[1,1,1,], d(l): [1,1,1,], D(l): [1,1,1,], %: [1,1,1,1]}
    flush(f,2)
    l_list = []; dl_list = []; Dl_list = []; percent_list = []
    for i in range(int(LMAX) + 1):
        temp_list = ut.getfloat(f)
        l_list.append(int(temp_list[0]))
        dl_list.append(temp_list[1])
        Dl_list.append(temp_list[2])
        percent_list.append(temp_list[3])
    update_dict = {'l':l_list, 'd(l)':dl_list, 'D(l)':Dl_list, '%':percent_list}
    return update_dict

def SITES_INFO(f):
    sites = []
    while 1:
        site_info = {}
        # site num
        try:
            site_num_line = f.readline()
        except:
            print("No more site information")
            return 0
        if "Site number" not in site_num_line:
            print("No more site information")
            return 0
        
        token = site_num_line.split(': ')
        site_num = int(token[1].strip())
        
        # BLATTS
        Si, WSA, SC = ut.getfloat(f)
        NSC, NVN = ut.getint(f)
        blatts = BLATTS(f)
        RSORT = ut.colonspacestring(f)
        rmesh = RMESH(f)
        setrot = SETROT(f)
        update = UPDATE(f)
        NVSF = int(ut.css(f))
        SIGMA_01 = ut.e1f(f)
        SIGMA_0NSR = ut.e1f(f)
        flush(f) # Volume 지우기
        VOL_INSCRIBED = ut.getfloat(f)[0]
        VOL_INTEGRATED = ut.getfloat(f)[0]
        VOL_SUMMED = ut.getfloat(f)[0]
        VOL_EXACT = ut.getfloat(f)[0]
        ERROR = ut.getfloat(f)[0]
        
        # 데이터 dict에 넣기
        site_info['site_num'] = site_num
        site_info['Si'] = Si; site_info['WSA'] = WSA; site_info['SC'] = SC
        site_info['NSC'] = NSC; site_info['NVN'] = NVN
        site_info['BLATTS'] = blatts; site_info['RSORT'] = RSORT
        site_info['RMESH'] = rmesh; site_info['SETROT'] = setrot
        site_info['UPDATE'] = update; site_info['NVSF'] = NVSF
        site_info['SIGMA_01'] = SIGMA_01; site_info['SIGMA_0NSR'] = SIGMA_0NSR
        site_info['VOL_INSCRIBED'] = VOL_INSCRIBED; site_info['VOL_INTEGRATED'] = VOL_INTEGRATED
        site_info['VOL_SUMMED'] = VOL_SUMMED; site_info['VOL_EXACT'] = VOL_EXACT
        site_info['ERROR'] = ERROR
        
        sites.append(site_info)
    return sites    ### 내용 리스트 내보내기
    


In [6]:
filename="lat_bcc.prn"
### 빈줄 제거
with open(filename, 'r') as f:
    lines = f.readlines()
lines = [line for line in lines if line.strip()]
with open('temp_'+filename, 'w') as f:
    f.writelines(lines)
    
### 데이터 읽기
with open('temp_'+filename, 'r') as f:
    # FHNDLR
    time_start = FHNDLR1(f)
    FOR001 = FHNDLR2(f)
    EMTO, branch, hash_key, compile_on, OS, CPU, compiler, library = FHNDLR3(f)
        
    # print(time_start, FOR001, EMTO, branch, hash_key, compile_on, OS, CPU, compiler, library)
    
    NSR, LMAX, NFI, IVEF = INPUT(f)
    slope_matrices, KW2 = TRNSFM(f)
    # print(NSR, LMAX, NFI, IVEF, slope_matrices, KW2)
    
    site_info = SITES_INFO(f)
    # print(site_info)
    
# Combine entities
Output = {
    'FHNDLR': {
        'time_start': [time_start, None],
        'FOR001': [FOR001, None],
        'EMTO': [EMTO, None],
        'branch': [branch, None],
        'hash_key': [hash_key, None],
        'compile_on': [compile_on, None],
        'OS': [OS, None],
        'CPU': [CPU, None],
        'compiler': [compiler, None],
        'library': [library, None]
    },
    'INPUT': {
        'NSR': [NSR, None],
        'LMAX': [LMAX, None],
        'NFI': [NFI, None],
        'IVEF': [IVEF, None]
    },
    'TRNSFM': {
        'slope_matrices': [slope_matrices, None],
        'KW2': [KW2, None]
    },
    'SITES_INFO': site_info
}

with open('output.json', 'w') as f:
    json.dump(Output, f, indent=4)
    
### temp 파일 지우기

           V(tetra) =  0.50000000  V(atom)  =  0.50000000

No more site information


In [7]:
Output

{'FHNDLR': {'time_start': [1638473640.0, None],
  'FOR001': ['../../kstr/lat_bcc/smx/lat_bcc.tfh', None],
  'EMTO': ['not available', None],
  'branch': ['not available', None],
  'hash_key': ['not available', None],
  'compile_on': ['Thu Apr  9 21:06:12 KST 2020', None],
  'OS': ['Linux 3.10.0-957.el7.x86_64 x86_64 GNU/Linux', None],
  'CPU': ['Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz', None],
  'compiler': ['ifort (IFORT) 18.0.5 20180823', None],
  'library': ['mkl', None]},
 'INPUT': {'NSR': [129, None],
  'LMAX': [30, None],
  'NFI': [11, None],
  'IVEF': [3, None]},
 'TRNSFM': {'slope_matrices': ['lat_bcc', None], 'KW2': [0.0, None]},
 'SITES_INFO': [{'site_num': 1,
   'Si': 0.433013,
   'WSA': 0.492373,
   'SC': 0.559017,
   'NSC': 24,
   'NVN': 14,
   'BLATTS': [{'plane_num': 1,
     'plane_info': [0.5, 0.5, -0.5, 0.8660254],
     'point_num': 6,
     'point_info': [[0.5, 0.0, -0.25, 0.55901699],
      [0.25, 0.0, -0.5, 0.55901699],
      [0.0, 0.25, -0.5, 0.55901699],
      [0

In [11]:
# string = 'INPUT     NSR =  129 LMAX=   30 NFI =   11 IVEF=    3'
# string = 'NSRI( 1)=          71 DSRI( 1)=  0.00095696'
string = 'A=     =  1.000000 B=     =  1.000000 C=     =  1.000000'
float_list = [float(s) for s in re.findall(r"[-+]?\d*\.\d+|\d+", string)]
str_list = [s for s in string.split() if not re.match(r"[-+]?\d*\.\d+|\d+", s)]
res = [s for s in string.split() if not s.replace('.','',1).isdigit()]
# 해당 list에서 '=' 또는 ':' 제거 하기
str_list_2 = [s for s in str_list if s != '=' and s != ':']
str_list_3 = [s.replace('=','') for s in str_list_2]
token = string.split('=')[1:3]
str_list_4 = list(map(lambda x:x[:3],token))
# str_list = re.split('=|\s', string)
# print(str_list)
# int_list = [int(s) for s in str_list if s.isdigit()]
# print(int_list)
print(float_list)
print(str_list)
print(str_list_2)
print(str_list_3)
# print(res)
# print(str_list_4)

[1.0, 1.0, 1.0]
['A=', '=', 'B=', '=', 'C=', '=']
['A=', 'B=', 'C=']
['A', 'B', 'C']
